In [5]:
import mysql.connector
from mysql.connector import Error

In [ ]:
from connection import create_connection # used to connect the database

def check_connection(conn):
    if conn is None or not conn.is_connected():
        print("Connection lost. Reconnecting...")
        conn = create_connection()
    return conn

def show_menu():
    print("\n====== Game Store Management ======")
    print("1. Register New Customer")
    print("2. View All Games")
    print("3. Place an Order")
    print("4. Submit Review")
    print("5. Employee Management")
    print("6. Add New Game")
    print("7. Exit")


def register_customer(conn, cursor):
    try:
        first = input("First Name: ")
        last = input("Last Name: ")
        email = input("Email: ")
        phone = input("Phone Number: ")
        address = input("Address: ")

        cursor.execute("""
            INSERT INTO Customers (first_name, last_name, email, phone_number, address)
            VALUES (%s, %s, %s, %s, %s)
        """, (first, last, email, phone, address))
        conn.commit()
        print("Customer registered successfully!")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        conn.rollback()

def view_games(conn, cursor):
    conn = check_connection(conn)
    print("\n-- Available Games --")
    cursor.execute("SELECT game_id, title, platform, price, stock_quantity FROM Games")
    games = cursor.fetchall()
    for game in games:
        print(f"ID: {game[0]}, Title: {game[1]}, Platform: {game[2]}, Price: ${game[3]}, Stock: {game[4]}")

def insert_game(conn, cursor):
    try:
        title = input("Enter the title of the game: ")
        platform = input("Enter the platform of the game: ")
        price = float(input("Enter the price of the game: "))
        stock_quantity = int(input("Enter the stock quantity of the game: "))

        cursor.execute("""
            INSERT INTO Games (title, platform, price, stock_quantity)
            VALUES (%s, %s, %s, %s)
        """, (title, platform, price, stock_quantity))
        conn.commit()
        print(f"✅ Game '{title}' added successfully!")
    except Error as e:
        print(f"Failed to insert game: {e}")
        conn.rollback()

def place_order(conn, cursor):
    print("\n-- Place New Order --")
    customer_id = input("Customer ID: ")

    cursor.execute("SELECT * FROM Customers WHERE customer_id = %s", (customer_id,))
    if cursor.fetchone() is None:
        print("Customer not found.")
        return

    view_games(conn, cursor)
    order_items = []
    total_price = 0

    while True:
        title = input("Enter Game Title to Order (or type 'done' to finish): ")
        if title.lower() == 'done':
            break
        quantity = int(input("Quantity: "))

        cursor.execute("""
            SELECT game_id, title, price, stock_quantity, platform
            FROM Games
            WHERE LOWER(title) = LOWER(%s)
        """, (title,))
        game = cursor.fetchone()

        if not game:
            print("No game found with that title.")
            continue

        print(f"\n-- Game Found --")
        print(f"ID: {game[0]}, Title: {game[1]}, Platform: {game[4]}, Price: ${game[2]}, Stock: {game[3]}")

        if quantity > game[3]:
            print("Not enough stock.")
            continue

        total_price += game[2] * quantity
        order_items.append({
            "game_id": game[0],
            "title": game[1],
            "platform": game[4],
            "quantity": quantity,
            "price": game[2]
        })

    if not order_items:
        print("No valid games selected, order canceled.")
        return

    cursor.execute("""
        INSERT INTO Orders (customer_id, total_amount, payment_method, order_status)
        VALUES (%s, %s, %s, %s)
    """, (customer_id, total_price, 'Cash', 'Completed'))
    order_id = cursor.lastrowid
    conn.commit()

    for item in order_items:
        cursor.execute("""
            INSERT INTO Order_Items (order_id, game_id, quantity, price_per_unit)
            VALUES (%s, %s, %s, %s)
        """, (order_id, item['game_id'], item['quantity'], item['price']))

        cursor.execute("""
            UPDATE Games
            SET stock_quantity = stock_quantity - %s
            WHERE game_id = %s
        """, (item['quantity'], item['game_id']))

    conn.commit()

    print("\nOrder Summary:")
    for item in order_items:
        print(f"- {item['title']} ({item['platform']}) x{item['quantity']} @ ${item['price']} each")

    print(f"Total Cost: ${total_price:.2f}")
    print("Order placed successfully!")


def submit_review(conn, cursor):
    print("\n-- Submit Review --")
    customer_id = input("Customer ID: ")

    cursor.execute("SELECT * FROM Customers WHERE customer_id = %s", (customer_id,))
    if cursor.fetchone() is None:
        print("Customer not found.")
        return

    view_games(conn, cursor)
    game_id = input("Enter Game ID to Review: ")
    rating = int(input("Rating (1 to 5): "))
    comment = input("Comment: ")

    cursor.execute("""
        INSERT INTO Reviews (customer_id, game_id, rating, comment)
        VALUES (%s, %s, %s, %s)
    """, (customer_id, game_id, rating, comment))
    conn.commit()
    print("Review submitted successfully!")

def employee_management(conn, cursor):
    print("\n-- Employee Management --")
    action = input("Enter 'view' to view employees, or 'add' to add a new employee: ").lower()

    if action == 'view':
        cursor.execute("SELECT employee_id, first_name, last_name, position, salary FROM Employees")
        employees = cursor.fetchall()
        for emp in employees:
            print(f"ID: {emp[0]}, Name: {emp[1]} {emp[2]}, Position: {emp[3]}, Salary: ${emp[4]}")
    elif action == 'add':
        first = input("First Name: ")
        last = input("Last Name: ")
        position = input("Position: ")
        email = input("Email: ")
        phone = input("Phone Number: ")
        salary = float(input("Salary: "))

        cursor.execute("""
            INSERT INTO Employees (first_name, last_name, position, email, phone_number, hire_date, salary)
            VALUES (%s, %s, %s, %s, %s, CURRENT_TIMESTAMP, %s)
        """, (first, last, position, email, phone, salary))
        conn.commit()
        print("Employee added successfully!")

def main():
    conn = create_connection()
    if conn is None:
        print("Could not connect to the database.")
        return
    cursor = conn.cursor()

    while True:
        show_menu()
        choice = input("Choose an option: ")

        if choice == "1":
            register_customer(conn, cursor)
        elif choice == "2":
            view_games(conn, cursor)
        elif choice == "3":
            place_order(conn, cursor)
        elif choice == "4":
            submit_review(conn, cursor)
        elif choice == "5":
            employee_management(conn, cursor)
        elif choice == "6":
            insert_game(conn, cursor)
        elif choice == "7":
            print("👋 Exiting...")
            cursor.close()
            conn.close()
            break
        else:
            print("Invalid choice, please try again.")

if __name__ == "__main__":
    main()


Connection established

====== Game Store Management ======
1. Register New Customer
2. View All Games
3. Place an Order
4. Submit Review
5. Employee Management
6. Add New Game
7. Exit


Customer registered successfully!

====== Game Store Management ======
1. Register New Customer
2. View All Games
3. Place an Order
4. Submit Review
5. Employee Management
6. Add New Game
7. Exit

-- Available Games --
ID: 1, Title: Elden Ring, Platform: pc, ps5, xbox, Price: $60.00, Stock: 110
ID: 2, Title: GTA 5, Platform: pc, ps5, xbox, Price: $70.00, Stock: 198
ID: 3, Title: Far Cry 5, Platform: pc, ps5, ps4, xbox, Price: $50.00, Stock: 59
ID: 4, Title: God Of War Ragnarok, Platform: ps5, Price: $65.00, Stock: 80
ID: 5, Title: Devil May Cry 5, Platform: pc, ps5, xbox, Price: $50.00, Stock: 30
ID: 6, Title: Need For Speed Heat, Platform: pc, ps5, xbox, Price: $40.00, Stock: 40
ID: 7, Title: Call Of Duty Black Ops 6, Platform: pc, ps5, xbox, Price: $50.00, Stock: 120

====== Game Store Management ======
1. Register New Customer
2. View All Games
3. Place an Order
4. Submit Review
5. Employee Management
6. Add New Game
7. Exit

-- Place New Order --

-- Available Games --
ID: 1, Title